In [9]:
import numpy as np
import pandas as pd

In [17]:
dataset1 = pd.read_csv('DataSet\Data2_Hate_comments.csv')
dataset2 = pd.read_csv('DataSet\Data1_cyberbullying_tweets.csv')

In [18]:
dataset1.head()

,Unnamed: 0,file_id,user_id,subforum_id,num_contexts,label,content
0,0,12834217_1,572066,1346,0,noHate,"As of March 13th , 2014 , the booklet had been..."
1,1,12834217_2,572066,1346,0,noHate,In order to help increase the booklets downloa...
2,2,12834217_3,572066,1346,0,noHate,( Simply copy and paste the following text int...
3,3,12834217_4,572066,1346,0,hate,Click below for a FREE download of a colorfull...
4,4,12834217_5,572066,1346,0,noHate,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...


In [19]:
dataset2.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [20]:
data1 = dataset1[(dataset1['label']!='relation' ) & (dataset1['label']!='idk/skip')]

In [21]:
dataset2['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [22]:
data2=dataset2[dataset2['cyberbullying_type']!='other_cyberbullying']

In [23]:
data1['label'].value_counts()

noHate    9507
hate      1196
Name: label, dtype: int64

In [24]:
data2 = data2.rename(columns={'cyberbullying_type': 'label','tweet_text':'content'})

In [25]:
Result_map1 = {'noHate': 0, 'hate': 1}
data1['label'] = data1['label'].map(Result_map1)
Result_map2 = {'not_cyberbullying': 0, 'religion': 1,'age': 1,'gender': 1,'ethnicity': 1}
data2['label'] = data2['label'].map(Result_map2)

C:\Users\chara\AppData\Local\Temp\ipykernel_23468\3299201987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['label'] = data1['label'].map(Result_map1)


In [26]:
modified_data= pd.concat([data1, data2])

In [27]:
from sklearn.utils import resample
majority_class = modified_data[modified_data['label'] == 1]
minority_class = modified_data[modified_data['label'] == 0]
print(majority_class.shape,minority_class.shape)

(33120, 7) (17452, 7)


In [28]:
majority_downsampled = resample(majority_class,
                                replace=False,
                                n_samples=23000,
                                random_state=42)

minority_upsampled = resample(minority_class,
                                replace=True,
                                n_samples=20000,
                                random_state=42)

df_balanced = pd.concat([minority_upsampled, majority_downsampled])

data = df_balanced.sample(frac=1, random_state=42)

In [29]:
data.shape

(43000, 7)

In [30]:
data['label'].value_counts()

1    23000
0    20000
Name: label, dtype: int64

In [31]:
x=data.iloc[:,-1].values
y=data.iloc[:, -2].values

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)


### above is the code for data analysis 
### Below is the code for prediction

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

from keras.models import Sequential
from keras.layers import Dense, Dropout

In [35]:
def create_model():
    model = Sequential()
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [36]:
from functions import text_filter,to_dense

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:
x=text_filter(X_train)
cv = CountVectorizer(max_features = 3000)
x = cv.fit_transform(x)
x=to_dense(x)
model = create_model()
model.fit(x,y_train,epochs=10,batch_size=32)

Epoch 1/10
1075/1075 [==============================] - 4s 3ms/step - loss: 0.2742 - accuracy: 0.8891
Epoch 2/10
1075/1075 [==============================] - 3s 3ms/step - loss: 0.1713 - accuracy: 0.9390
Epoch 3/10
1075/1075 [==============================] - 3s 3ms/step - loss: 0.1374 - accuracy: 0.9531
Epoch 4/10
1075/1075 [==============================] - 3s 3ms/step - loss: 0.1134 - accuracy: 0.9613
Epoch 5/10
1075/1075 [==============================] - 3s 3ms/step - loss: 0.0929 - accuracy: 0.9683
Epoch 6/10
1075/1075 [==============================] - 4s 3ms/step - loss: 0.0789 - accuracy: 0.9735
Epoch 7/10
1075/1075 [==============================] - 3s 3ms/step - loss: 0.0683 - accuracy: 0.9776
Epoch 8/10
1075/1075 [==============================] - 3s 3ms/step - loss: 0.0603 - accuracy: 0.9808
Epoch 9/10
1075/1075 [==============================] - 3s 3ms/step - loss: 0.0544 - accuracy: 0.9817
Epoch 10/10
1075/1075 [==============================] - 3s 3ms/step - loss: 0.048

In [96]:
with open('Trained models\Count_vectors.pkl', 'wb') as f:
    pickle.dump(cv, f)

In [97]:
model.save('Trained models\\annmodel_3000.h5')

In [24]:

X=text_filter(X_test)
X=cv.transform(X)
X=to_dense(X)


In [ ]:
res=model.predict(X)
y_pred=(res > 0.4).astype(int)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
k = accuracy_score(y_test, y_pred)
print(k)


In [37]:
pipeline = Pipeline([
    ('text_filter', FunctionTransformer(text_filter)),
    ('count_vectorizer', cv),
    ('to_dense', FunctionTransformer(to_dense)),
    ('model', model)
])

NameError: name 'cv' is not defined

In [54]:
pipeline.predict(X_train)

1075/1075 [==============================] - 2s 2ms/step


array([[1.0000000e+00],
       [9.9962485e-01],
       [1.4430302e-06],
       ...,
       [9.0573990e-04],
       [1.0000000e+00],
       [1.0000000e+00]], dtype=float32)

In [30]:
score = pipeline.evaluate(X_test, y_test, batch_size=128)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

AttributeError: 'Pipeline' object has no attribute 'evaluate'

In [1]:
from functions import text_filter,to_dense

pipeline = Pipeline([
    ("filter",FunctionTransformer(text_filter)),
    ("vectorizer", cv),
    ("to_dense", FunctionTransformer(to_dense, accept_sparse=True)),
    ("model",model)
])
pipeline.fit(X_train, y_train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'Pipeline' is not defined

### pipeline for ml models

In [276]:
pipeline2 = make_pipeline(
    FunctionTransformer(text_filter),
    CountVectorizer(max_features=2000),
    FunctionTransformer(to_dense, accept_sparse=True),
    GaussianNB()
)
pipeline2.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function text_filter at 0x0000023DA5FFC700>)),
                ('countvectorizer', CountVectorizer(max_features=2000)),
                ('functiontransformer-2',
                 FunctionTransformer(accept_sparse=True,
                                     func=<function to_dense at 0x0000023DE744F8B0>)),
                ('gaussiannb', GaussianNB())])

### Accuracy Checker for the Pipeline

In [2]:
y_pred_pipeline=pipeline.predict(X_test)

NameError: name 'pipeline' is not defined

In [1]:
y_pred=(y_pred > 0.4).astype(int)

NameError: name 'y_pred' is not defined

In [62]:
print(y_pred)

[[1]
 [0]
 [1]
 ...
 [1]
 [0]
 [0]]


In [63]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
k = accuracy_score(y_test, y_pred)
print(k)

[[3815  189]
 [ 326 4270]]
0.9401162790697675


In [68]:
res=pipeline.predict(["you are a bad asshole"])

1/1 [==============================] - 0s 19ms/step


In [46]:
res = model.predict(test)

1/1 [==============================] - 0s 28ms/step


In [69]:
print(res)

[[0.27383068]]


### Saving the model for use

In [71]:
import joblib

In [72]:
joblib.dump(pipeline, f"ANN_3000_{k}.joblib")

INFO:tensorflow:Assets written to: ram://99140fa2-adb8-4f5f-8c44-49f69c9fd985/assets


['ANN_3000_0.9401162790697675.joblib']